In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, LassoCV, RidgeCV, ElasticNetCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, FunctionTransformer, Imputer
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
#read in both
ames_data = pd.read_csv("train.csv") 
#test = pd.read_csv('test.csv')

ames_data.columns = ames_data.columns.map(lambda x: str(x.replace(' ', '_')))

missing_data = ames_data.isnull().sum()

fig, ax = plt.subplots(figsize=(15,15))
missing_data.plot(kind = 'bar')

ames_data = ames_data.drop(["Alley",'Fireplace_Qu','Pool_QC','Fence', 'Misc_Feature'], axis = 1)

missing_data = ames_data.isnull().sum()

fig, ax = plt.subplots(figsize=(13,13))
missing_data.plot(kind = 'bar')

In [ ]:
ames_data = ames_data.dropna(axis = 0, how = 'any')

In [3]:
def string_cleaning(df):
    return df.replace({'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1,'Av': 3, 'Mn': 2, 'No': 1,\
                            'GLQ': 5, 'ALQ': 4, 'BLQ': 3, 'Rec': 4, 'LwQ': 2, 'Unf': 1, 'Typ': 5, 'Min1': 4,\
                            'Min2': 4, 'Mod': 3, 'Maj1': 2, 'Maj2': 2, 'Sal': 0,'Fin': 3, 'RFn': 2, 'Unf': 1,\
                            'AllPub': 1, 'Pave': 1, 'Grvl': 0, 'AllPub': 4, 'NoSewr':3, 'NoSeWa':2, "ELO":1 }, inplace = True)
string_cleaning(ames_data)

string_cleaning_tf = FunctionTransformer(string_cleaning, validate=False)
ames_data.head()

In [6]:
ames_data["Total_SF"] = ames_data['Gr_Liv_Area'] + ames_data["Total_Bsmt_SF"] + ames_data["Garage_Area"] + ames_data['Wood_Deck_SF']\
    + ames_data["Open_Porch_SF"] + ames_data['Enclosed_Porch'] + ames_data['3Ssn_Porch'] + ames_data['Screen_Porch']\
    + ames_data['Pool_Area']

ames_data["Total_Bathrooms"] = ames_data['Bsmt_Full_Bath'] + ames_data['Bsmt_Half_Bath'] + ames_data['Full_Bath'] + ames_data['Half_Bath']

ames_data['Quality_Average_5'] = (ames_data['Exter_Qual'] + ames_data['Exter_Cond'] + ames_data['Bsmt_Qual'] + ames_data['Bsmt_Cond'] + ames_data['Bsmt_Exposure'] + ames_data['BsmtFin_Type_1'] + ames_data['BsmtFin_Type_2'] + ames_data['Heating_QC'] + ames_data['Kitchen_Qual'] + ames_data['Garage_Cond'] + ames_data['Garage_Qual']).mean()

ames_data['Quality_Average_10'] = (ames_data["Overall_Qual"] + ames_data['Overall_Cond']).mean()

In [7]:
ames_data = pd.get_dummies(ames_data)

In [10]:
ames_data.head()
ames_data.shape


(1556, 218)

In [11]:
ames_data.corr()['SalePrice'].sort_values(ascending = False)

SalePrice                 1.000
Overall_Qual              0.807
Total_SF                  0.803
Exter_Qual                0.724
Gr_Liv_Area               0.708
Kitchen_Qual              0.700
Bsmt_Qual                 0.693
Garage_Cars               0.673
Garage_Area               0.672
1st_Flr_SF                0.641
Total_Bsmt_SF             0.636
Total_Bathrooms           0.612
Year_Built                0.567
Full_Bath                 0.561
Garage_Yr_Blt             0.550
TotRms_AbvGrd             0.547
Year_Remod/Add            0.547
Foundation_PConc          0.545
Garage_Finish             0.544
Mas_Vnr_Area              0.533
Neighborhood_NridgHt      0.476
Heating_QC                0.453
Fireplaces                0.445
Bsmt_Exposure             0.440
BsmtFin_SF_1              0.405
Lot_Area                  0.395
Sale_Type_New             0.375
Sale_Condition_Partial    0.372
Lot_Frontage              0.350
Exterior_1st_VinylSd      0.346
                          ...  
Sale_Typ